In [251]:
import pandas as pd
import numpy as np
#eplus_out_file = "/home/zhiangz/Documents/HVAC-RL-Control/src/eplus-env/eplus_env/envs/eplus_models/iw_v96/learning/idf/realWeather_base_out/eplusout.csv";
eplus_out_file = '/home/zhiangz/Documents/HVAC-RL-Control/src/iw_ctrl_runs_v2/88/Eplus-env-IW-realWeather-v9706-res1/Eplus-env-sub_run41/output/eplusout.csv'
eplus_out_file = '~/Documents/HVAC-RL-Control/src/eplus-env/eplus_env/envs/eplus_models/iw_v96/learning/idf/realWeather_base_bas_out/eplusout.csv'
occupant_count_colname = 'EMS:IW Occupy Mode Flag [](TimeStep)'
ppd_colname = 'EMS:IW Average PPD [%](TimeStep)'
energy_colname = 'EMS:IW Calculated Heating Demand [KW](TimeStep)'
stptbylogic_colname = 'EMS:IW Effective IAT Setpoint by Logics [C](TimeStep)'
iat_colname = 'EMS:IW North Zone Average Temperature [C](TimeStep)'
timestep = 5 # Minutes

In [252]:
def getFullColname(colnames, tgtname):
    for colname in colnames:
        if tgtname in colname:
            return colname;

In [253]:
df = pd.read_csv(eplus_out_file)
df_colnames = list(df.columns.values)
occupant_count_colname = getFullColname(df_colnames, occupant_count_colname)
ppd_colname = getFullColname(df_colnames, ppd_colname)
energy_colname = getFullColname(df_colnames, energy_colname)
stptbylogic_colname = getFullColname(df_colnames, stptbylogic_colname)
iat_colname = getFullColname(df_colnames, iat_colname);

In [254]:
occp_ppd = df[ppd_colname].loc[df[occupant_count_colname] > 0]
valid_energy = df[energy_colname].loc[np.isnan(df[energy_colname]) != 1]
iat_stpt_diff = df[iat_colname].loc[df[occupant_count_colname] == 0] - df[stptbylogic_colname].loc[df[occupant_count_colname] == 0]

In [255]:
mean_occp_ppd = np.mean(occp_ppd)
std_occp_ppd = np.std(occp_ppd)
total_energy = np.sum(valid_energy) * (timestep/60.0)
mean_iat_notmet = np.mean(iat_stpt_diff)
std_iat_notmet = np.std(iat_stpt_diff);

In [256]:
print ('Total HVAC energy consumption is %d kHW, mean occupied PPD is %0.02f%%, std occupied PPD is %0.02f%%'\
       ', mean IAT not met during unoccupied hour is %0.02f, std IAT not met during unoccupied hour is %0.02f.'
      %(total_energy, mean_occp_ppd, std_occp_ppd, mean_iat_notmet, std_iat_notmet))

Total HVAC energy consumption is 43709 kHW, mean occupied PPD is 9.46%, std occupied PPD is 5.59%, mean IAT not met during unoccupied hour is 0.50, std IAT not met during unoccupied hour is 1.18.


In [257]:
np.histogram(occp_ppd)

(array([11092,  2524,   883,   334,   237,   136,    40,    24,    20,    32]),
 array([  5.21069202,  10.02746179,  14.84423156,  19.66100133,
         24.47777111,  29.29454088,  34.11131065,  38.92808042,
         43.74485019,  48.56161996,  53.37838973]))